### Links
- PyTorch MNIST example - https://github.com/pytorch/examples/blob/master/mnist/main.py
- pytorch cifar10 code - https://github.com/kuangliu/pytorch-cifar/blob/master/main.py
- LeNet for CIFAR-10 - https://github.com/kuangliu/pytorch-cifar/blob/master/models/lenet.py

### My notes
- My Google Doc - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
- Trello - https://trello.com/b/ycCGjD3A/mobile-dl
- Using webcam - https://docs.google.com/document/d/17BVx5yioNaV11mip4U9gRAO7wLIJf-aOpIITJPjqeks/edit#
- Installing pytorch - https://docs.google.com/document/d/1NfLbuhoOxonzS9FEeooyqpB90-62Gat-uZ2DOTaWRdg/edit#


In [1]:
import argparse
import math
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [3]:
# Source: https://github.com/kuangliu/pytorch-cifar/blob/master/models/mobilenet.py

class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
#         print(out.size())
        out = F.relu(self.bn2(self.conv2(out)))
#         print(out.size())
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [(48,2), (64,2), (128,2), (256,2), 256]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layers = self._make_layers(in_planes=16)
        self.linear = nn.Linear(256, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
#         print(out.size())
        out = self.layers(out)
#         print(out.size())
        out = F.avg_pool2d(out, 2)
#         print(out.size())
        out = out.view(out.size(0), -1)
#         print(out.size())
        out = self.linear(out)
#         print(out.size())
        return out


In [4]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')


_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='how many batches to wait before logging training status', metavar='N')

In [5]:
args = parser.parse_args(args=["--epochs=10"])
args

Namespace(batch_size=64, epochs=10, log_interval=10, lr=0.01, momentum=0.5, no_cuda=False, seed=1, test_batch_size=1000)

In [6]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda

False

In [7]:
torch.manual_seed(args.seed)

In [8]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [9]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs

{}

In [10]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
#                        transforms.Resize((32, 32)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.Resize((32, 32)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [11]:
model = MobileNet().to(device)
model

MobileNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Block(
      (conv1): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): B

In [12]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [13]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # to do: needed?
        optimizer.zero_grad()  # what does this do? zero-out any previous gradient info?
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            digits = int(math.log10(len(train_loader.dataset))) + 1
            print('Train Epoch: {} [{:05d}/{:05d} ({:02.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            

In [14]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)
    
    # save the model!
    # source for timestamp formatting: https://stackoverflow.com/q/10607688/781938
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    # source for f-string: https://stackoverflow.com/a/42562236/781938
    torch.save(
        model.state_dict(),
        f'{timestamp}-mnist_mobilenet-{epoch:05d}.pytorchmodel')


Train Epoch: 1 [00000/60000 (00%)]	Loss: -0.027056
Train Epoch: 1 [00640/60000 (01%)]	Loss: -0.360708
Train Epoch: 1 [01280/60000 (02%)]	Loss: -0.707185
Train Epoch: 1 [01920/60000 (03%)]	Loss: -1.043764
Train Epoch: 1 [02560/60000 (04%)]	Loss: -1.472964
Train Epoch: 1 [03200/60000 (05%)]	Loss: -1.841768
Train Epoch: 1 [03840/60000 (06%)]	Loss: -2.332770
Train Epoch: 1 [04480/60000 (07%)]	Loss: -2.791593
Train Epoch: 1 [05120/60000 (09%)]	Loss: -3.270411
Train Epoch: 1 [05760/60000 (10%)]	Loss: -4.022709
Train Epoch: 1 [06400/60000 (11%)]	Loss: -4.550189
Train Epoch: 1 [07040/60000 (12%)]	Loss: -5.370422
Train Epoch: 1 [07680/60000 (13%)]	Loss: -6.216835
Train Epoch: 1 [08320/60000 (14%)]	Loss: -7.565855
Train Epoch: 1 [08960/60000 (15%)]	Loss: -8.563569
Train Epoch: 1 [09600/60000 (16%)]	Loss: -10.175191
Train Epoch: 1 [10240/60000 (17%)]	Loss: -11.554020
Train Epoch: 1 [10880/60000 (18%)]	Loss: -12.635107
Train Epoch: 1 [11520/60000 (19%)]	Loss: -14.026346
Train Epoch: 1 [12160/60000

Train Epoch: 2 [35200/60000 (59%)]	Loss: -204192800.000000
Train Epoch: 2 [35840/60000 (60%)]	Loss: -190855024.000000
Train Epoch: 2 [36480/60000 (61%)]	Loss: -196809104.000000
Train Epoch: 2 [37120/60000 (62%)]	Loss: -197768160.000000
Train Epoch: 2 [37760/60000 (63%)]	Loss: -230236800.000000
Train Epoch: 2 [38400/60000 (64%)]	Loss: -346967424.000000
Train Epoch: 2 [39040/60000 (65%)]	Loss: -389362048.000000
Train Epoch: 2 [39680/60000 (66%)]	Loss: -274205472.000000
Train Epoch: 2 [40320/60000 (67%)]	Loss: -431194816.000000
Train Epoch: 2 [40960/60000 (68%)]	Loss: -513286784.000000
Train Epoch: 2 [41600/60000 (69%)]	Loss: -541571392.000000
Train Epoch: 2 [42240/60000 (70%)]	Loss: -619272896.000000
Train Epoch: 2 [42880/60000 (71%)]	Loss: -748308800.000000
Train Epoch: 2 [43520/60000 (72%)]	Loss: -776496576.000000
Train Epoch: 2 [44160/60000 (74%)]	Loss: -944330880.000000
Train Epoch: 2 [44800/60000 (75%)]	Loss: -957918528.000000
Train Epoch: 2 [45440/60000 (76%)]	Loss: -1277229312.000

Train Epoch: 3 [58880/60000 (98%)]	Loss: -1306882446721024.000000
Train Epoch: 3 [59520/60000 (99%)]	Loss: -1410198421897216.000000

Test set: Average loss: -1617007207005343.0000, Accuracy: 1135/10000 (11%)

Train Epoch: 4 [00000/60000 (00%)]	Loss: -1738940855877632.000000
Train Epoch: 4 [00640/60000 (01%)]	Loss: -1642117931728896.000000
Train Epoch: 4 [01280/60000 (02%)]	Loss: -1923532644679680.000000
Train Epoch: 4 [01920/60000 (03%)]	Loss: -2431691834523648.000000
Train Epoch: 4 [02560/60000 (04%)]	Loss: -2653914516160512.000000
Train Epoch: 4 [03200/60000 (05%)]	Loss: -2820465563271168.000000
Train Epoch: 4 [03840/60000 (06%)]	Loss: -3320549275074560.000000
Train Epoch: 4 [04480/60000 (07%)]	Loss: -3671620438720512.000000
Train Epoch: 4 [05120/60000 (09%)]	Loss: -4117230006566912.000000
Train Epoch: 4 [05760/60000 (10%)]	Loss: -4414696220262400.000000
Train Epoch: 4 [06400/60000 (11%)]	Loss: -5195522079981568.000000
Train Epoch: 4 [07040/60000 (12%)]	Loss: -5641115810136064.000000

Train Epoch: 5 [14080/60000 (23%)]	Loss: -1657092305279067881472.000000
Train Epoch: 5 [14720/60000 (25%)]	Loss: -1896116914852083007488.000000
Train Epoch: 5 [15360/60000 (26%)]	Loss: nan
Train Epoch: 5 [16000/60000 (27%)]	Loss: nan
Train Epoch: 5 [16640/60000 (28%)]	Loss: nan
Train Epoch: 5 [17280/60000 (29%)]	Loss: nan
Train Epoch: 5 [17920/60000 (30%)]	Loss: nan
Train Epoch: 5 [18560/60000 (31%)]	Loss: nan
Train Epoch: 5 [19200/60000 (32%)]	Loss: nan
Train Epoch: 5 [19840/60000 (33%)]	Loss: nan
Train Epoch: 5 [20480/60000 (34%)]	Loss: nan
Train Epoch: 5 [21120/60000 (35%)]	Loss: nan
Train Epoch: 5 [21760/60000 (36%)]	Loss: nan
Train Epoch: 5 [22400/60000 (37%)]	Loss: nan
Train Epoch: 5 [23040/60000 (38%)]	Loss: nan
Train Epoch: 5 [23680/60000 (39%)]	Loss: nan
Train Epoch: 5 [24320/60000 (41%)]	Loss: nan
Train Epoch: 5 [24960/60000 (42%)]	Loss: nan
Train Epoch: 5 [25600/60000 (43%)]	Loss: nan
Train Epoch: 5 [26240/60000 (44%)]	Loss: nan
Train Epoch: 5 [26880/60000 (45%)]	Loss: nan
T

Train Epoch: 7 [08320/60000 (14%)]	Loss: nan
Train Epoch: 7 [08960/60000 (15%)]	Loss: nan
Train Epoch: 7 [09600/60000 (16%)]	Loss: nan
Train Epoch: 7 [10240/60000 (17%)]	Loss: nan
Train Epoch: 7 [10880/60000 (18%)]	Loss: nan
Train Epoch: 7 [11520/60000 (19%)]	Loss: nan
Train Epoch: 7 [12160/60000 (20%)]	Loss: nan
Train Epoch: 7 [12800/60000 (21%)]	Loss: nan
Train Epoch: 7 [13440/60000 (22%)]	Loss: nan
Train Epoch: 7 [14080/60000 (23%)]	Loss: nan
Train Epoch: 7 [14720/60000 (25%)]	Loss: nan
Train Epoch: 7 [15360/60000 (26%)]	Loss: nan
Train Epoch: 7 [16000/60000 (27%)]	Loss: nan
Train Epoch: 7 [16640/60000 (28%)]	Loss: nan
Train Epoch: 7 [17280/60000 (29%)]	Loss: nan
Train Epoch: 7 [17920/60000 (30%)]	Loss: nan
Train Epoch: 7 [18560/60000 (31%)]	Loss: nan
Train Epoch: 7 [19200/60000 (32%)]	Loss: nan
Train Epoch: 7 [19840/60000 (33%)]	Loss: nan
Train Epoch: 7 [20480/60000 (34%)]	Loss: nan
Train Epoch: 7 [21120/60000 (35%)]	Loss: nan
Train Epoch: 7 [21760/60000 (36%)]	Loss: nan
Train Epoc

Train Epoch: 9 [03200/60000 (05%)]	Loss: nan
Train Epoch: 9 [03840/60000 (06%)]	Loss: nan
Train Epoch: 9 [04480/60000 (07%)]	Loss: nan
Train Epoch: 9 [05120/60000 (09%)]	Loss: nan
Train Epoch: 9 [05760/60000 (10%)]	Loss: nan
Train Epoch: 9 [06400/60000 (11%)]	Loss: nan
Train Epoch: 9 [07040/60000 (12%)]	Loss: nan
Train Epoch: 9 [07680/60000 (13%)]	Loss: nan
Train Epoch: 9 [08320/60000 (14%)]	Loss: nan
Train Epoch: 9 [08960/60000 (15%)]	Loss: nan
Train Epoch: 9 [09600/60000 (16%)]	Loss: nan
Train Epoch: 9 [10240/60000 (17%)]	Loss: nan
Train Epoch: 9 [10880/60000 (18%)]	Loss: nan
Train Epoch: 9 [11520/60000 (19%)]	Loss: nan
Train Epoch: 9 [12160/60000 (20%)]	Loss: nan
Train Epoch: 9 [12800/60000 (21%)]	Loss: nan
Train Epoch: 9 [13440/60000 (22%)]	Loss: nan
Train Epoch: 9 [14080/60000 (23%)]	Loss: nan
Train Epoch: 9 [14720/60000 (25%)]	Loss: nan
Train Epoch: 9 [15360/60000 (26%)]	Loss: nan
Train Epoch: 9 [16000/60000 (27%)]	Loss: nan
Train Epoch: 9 [16640/60000 (28%)]	Loss: nan
Train Epoc

KeyboardInterrupt: 